In [2]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import featuretools as ft
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from random import sample 

%matplotlib inline
RSEED = 50

# Load Original Features

In [3]:
feat_num = 469
df_total = pd.read_csv('./data/features%s.csv'%(feat_num))

In [4]:
df_train = df_total[df_total['isFraud'].notnull()]
df_train.shape

(590540, 469)

# Sample Train Data

In [5]:
df_train_sample=df_train.sample(n=100000,random_state=RSEED)
df_train_sample.shape

(100000, 469)

In [6]:
df_train_sample[df_train_sample.isFraud==0].shape

(96539, 469)

In [7]:
df_train_sample[df_train_sample.isFraud==1].shape

(3461, 469)

# Prepare Data

In [44]:
labels_train = df_train_sample['isFraud']
features_train = df_train_sample.drop(columns = ['isFraud', 'TransactionID'])
features_train.shape

(100000, 467)

In [45]:
categorical_raw = ['ProductCD', 'card2', 'card3', 'card4', 'card5','card6',
              'addr1','addr2','P_email','R_email','M1','M2','M3',
              'M4','M5','M6','M7','M8','M9','DeviceType','DeviceInfo','dow','hour',
              'Device_name','Device_version','screen_width','screen_height',
               'P_email_suffix','R_email_suffix','id_30_OS','id_30_version']
ids = [ 'id_%s'%(i) for i in range(12,39)]
categorical_raw = categorical_raw + ids

In [55]:
features_train[categorical].nunique()

id_23               4
id_13              39
id_28               3
ProductCD           5
id_16               3
M2                  3
id_32               4
id_15               4
id_30_version      42
id_21             183
id_22              15
id_37               3
card6               5
M6                  3
id_27               3
screen_height      72
id_12               3
id_19             417
screen_width       79
id_30_OS            8
addr2              34
id_38               3
id_31              25
M8                  3
addr1             175
card2             500
card4               5
id_29               3
id_34               5
id_36               3
Device_name        14
id_14              21
id_26              65
id_17              70
R_email            46
card3              73
dow                 7
M7                  3
M5                  3
id_20             241
id_35               3
hour               24
M9                  3
card5              83
id_18              13
R_email_su

# Select Features

In [46]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
          #'is_unbalance':True
          #'scale_pos_weight':9
         }

In [47]:
def train_selector(train_num,features_train,labels_train,categorical):
    train_set = lgb.Dataset(features_train.iloc[0:train_num,:], label=labels_train.values[0:train_num],
                       categorical_feature=categorical)
    valid_set = lgb.Dataset(features_train.iloc[train_num:,:], label=labels_train.values[train_num:],
                       categorical_feature=categorical)
    valid_results = {}
    model = lgb.train(params,train_set,num_boost_round = 10000, 
                   valid_sets = [train_set, valid_set],
                    verbose_eval=500,
                    early_stopping_rounds = 500,
                    evals_result=valid_results)
    return model,valid_results

## Train with all feature set

In [48]:
train_num = 80000
categorical = list(set(categorical_raw).intersection(features_train.columns))
model,valid_results = train_selector(train_num,features_train,labels_train,categorical)

/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996677	valid_1's auc: 0.918242
[1000]	training's auc: 0.999952	valid_1's auc: 0.924871
[1500]	training's auc: 1	valid_1's auc: 0.926299
[2000]	training's auc: 1	valid_1's auc: 0.926302
Early stopping, best iteration is:
[1628]	training's auc: 1	valid_1's auc: 0.926277


In [50]:
rm_col

'TransactionAmt_to_std_card1'

## Recursive Eliminate Features

In [49]:
best = 0.926277
rm_col = ''
for col in features_train.columns:
    print(col)
    features_train_temp = features_train.drop(col,axis=1)
    categorical_temp = list(set(categorical_raw).intersection(features_train_temp.columns))
    model,valid_results = train_selector(train_num,features_train_temp,labels_train,categorical_temp)
    pfm = max(valid_results['valid_1']['auc'])
    if pfm > best:
        print('remove %s improve auc from %s to %s this feature is endanger'%(col,best,pfm))
        best = pfm
        rm_col = col
        

TransactionDT


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996344	valid_1's auc: 0.917875
[1000]	training's auc: 0.999937	valid_1's auc: 0.925077
[1500]	training's auc: 1	valid_1's auc: 0.925968
[2000]	training's auc: 1	valid_1's auc: 0.926159
Early stopping, best iteration is:
[1723]	training's auc: 1	valid_1's auc: 0.925996
TransactionAmt


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996682	valid_1's auc: 0.917678
[1000]	training's auc: 0.999952	valid_1's auc: 0.925449
[1500]	training's auc: 1	valid_1's auc: 0.926479
[2000]	training's auc: 1	valid_1's auc: 0.92698
Early stopping, best iteration is:
[1662]	training's auc: 1	valid_1's auc: 0.926695
remove TransactionAmt improve auc from 0.926277 to 0.9269954966471345 this feature is endanger
ProductCD


...

Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996684	valid_1's auc: 0.917414
[1000]	training's auc: 0.999956	valid_1's auc: 0.925045
[1500]	training's auc: 1	valid_1's auc: 0.926429
[2000]	training's auc: 1	valid_1's auc: 0.926648
Early stopping, best iteration is:
[1640]	training's auc: 1	valid_1's auc: 0.926734
screen_width


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996697	valid_1's auc: 0.917584
[1000]	training's auc: 0.999956	valid_1's auc: 0.925164
[1500]	training's auc: 1	valid_1's auc: 0.926374
[2000]	training's auc: 1	valid_1's auc: 0.926589
Early stopping, best iteration is:
[1650]	training's auc: 1	valid_1's auc: 0.926453
screen_height


/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/jinpwa/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 500 rounds.
[500]	training's auc: 0.996639	valid_1's auc: 0.917488
[1000]	training's auc: 0.999954	valid_1's auc: 0.9249
[1500]	training's auc: 1	valid_1's auc: 0.926295
Early stopping, best iteration is:
[1426]	training's auc: 1	valid_1's auc: 0.926423
